In [1]:
# Basic setup for displaying bokeh plots in jupyter.
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
# Additional requirements for Stacked Bar plot.
from bokeh.core.properties import value
from bokeh.models import ColumnDataSource
output_notebook()

Loading BokehJS ...

In [2]:
# NumPy imports.
import numpy as np
import cPickle as pickle
import os
cwd = os.getcwd()
print(cwd)

/Users/kevin/dev/nml/paper_platform/scripts


In [43]:
# Raw data.
raws = pickle.load(open('2018-04-03-14-01-41-627705_singlerun_100.p', 'rb'))
print(type(raws))
print(len(raws))
print(raws[0])

<type 'list'>
100
{'/media/kevin/d640/enterobase_db/ESC_BA1121AA_AS.fasta': [{u'job_ptstx1_datastruct': [u'Tue, 03 Apr 2018 18:02:37 GMT', u'Tue, 03 Apr 2018 18:02:38 GMT', 1.0]}, {u'job_amr_dict': [u'Tue, 03 Apr 2018 18:03:30 GMT', u'Tue, 03 Apr 2018 18:03:30 GMT', 0.0]}, {u'job_ectyper_beautify_serotype': [u'Tue, 03 Apr 2018 18:01:57 GMT', u'Tue, 03 Apr 2018 18:01:58 GMT', 1.0]}, {u'job_ectyper_beautify_vf': [u'Tue, 03 Apr 2018 18:01:55 GMT', u'Tue, 03 Apr 2018 18:01:55 GMT', 0.0]}, {u'job_ectyper_datastruct_vf': [u'Tue, 03 Apr 2018 18:01:55 GMT', u'Tue, 03 Apr 2018 18:01:56 GMT', 1.0]}, {u'job_pteae': [u'Tue, 03 Apr 2018 18:01:56 GMT', u'Tue, 03 Apr 2018 18:03:28 GMT', 92.0]}, {u'job_ptstx1_dict': [u'Tue, 03 Apr 2018 18:02:37 GMT', u'Tue, 03 Apr 2018 18:02:37 GMT', 0.0]}, {u'job_ectyper_serotype': [u'Tue, 03 Apr 2018 18:01:52 GMT', u'Tue, 03 Apr 2018 18:01:57 GMT', 5.0]}, {u'job_pteae_dict': [u'Tue, 03 Apr 2018 18:03:28 GMT', u'Tue, 03 Apr 2018 18:03:28 GMT', 0.0]}, {u'job_ptstx1_be

In [66]:
# Tabulate runtimes per module.
parsed = {
    'qc':[],
    'databaseid':[],
    'serotype':[],
    'vf':[],
    'amr':[],
    'stx1':[],
    'stx2':[],
    'eae':[],
    'total': []
}
for outerd in raws:
    qc=0
    databaseid=0
    serotype=0
    vf=0
    amr=0
    stx1=0
    stx2=0
    eae=0
    total=0
    for d in outerd.values()[0]:
        if 'stx1' in d.keys()[0]:
            stx1 += d.values()[0][2]
        elif 'stx2' in d.keys()[0]:
            stx2 += d.values()[0][2]
        elif 'eae' in d.keys()[0]:
            eae += d.values()[0][2]
        elif 'vf' in d.keys()[0]:
            vf += d.values()[0][2]
        elif 'amr' in d.keys()[0]:
            amr += d.values()[0][2]
        elif 'serotype'in d.keys()[0]:
            serotype += d.values()[0][2]
        elif 'job_id' in d.keys()[0]:
            databaseid += d.values()[0][2]
        elif 'turtle' in d.keys()[0]:
            databaseid += d.values()[0][2]
        elif 'job_qc' in d.keys()[0]:
            qc += d.values()[0][2]
        elif 'total' in d.keys()[0]:
            total += d.values()[0][2]
    parsed['qc'].append(qc)
    parsed['databaseid'].append(databaseid)
    parsed['serotype'].append(serotype)
    parsed['vf'].append(vf)
    parsed['amr'].append(amr)
    parsed['stx1'].append(stx1)
    parsed['stx2'].append(stx2)
    parsed['eae'].append(eae)
    parsed['total'].append(total)

In [62]:
import numpy as np
from bokeh.models import ColumnDataSource
from bokeh.models.glyphs import Line

colormap = {'qc':'red',
            'databaseid':'green',
            'serotype':'blueviolet',
            'vf':'crimson',
            'amr':'cadetblue',
            'stx1':'darksalmon',
            'stx2':'darkorange',
            'eae':'darkgoldenrod',
            'total': 'blue'}
colors = [colormap[x] for x in parsed.keys() for i in range(len(parsed.values()[0]))]

x_runs = [i for l in parsed.values() for i,x in enumerate(l)]
y_times = [x for l in parsed.values() for x in l]

p = figure(title = "Timings for Individual Runs")
p.xaxis.axis_label = 'Run #'
p.yaxis.axis_label = 'Runtime (seconds)'

print(np.mean(parsed['total']))

lines_source = ColumnDataSource(data=dict(y=[np.mean(parsed['total']) for i in range(len(parsed['total']))], x=[i for i,x in enumerate(parsed['total'])]))
line = Line(x='x',y='y', line_color="#666699", line_width=2)

p.add_glyph(lines_source, line)

p.circle(x_runs, y_times,
         color=colors, fill_alpha=0.2, size=10)

show(p)

129.53


In [65]:
avgs = {k:np.mean(parsed[k]) for k in parsed}
print(avgs)

{'databaseid': 35.38, 'amr': 93.78, 'vf': 4.24, 'stx1': 4.26, 'stx2': 2.88, 'eae': 28.83, 'serotype': 6.24, 'qc': 0.69, 'total': 129.53}


In [4]:
# Data.
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries'] # list_sizes
years = ["2015", "2016", "2017"] # subtasks
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

source = ColumnDataSource(data=data)

In [5]:
# Plot.
p = figure(x_range=fruits,
           plot_height=350,
           title="Runtimes for Analysis Modules",
           x_axis_label="#Total Batch Size",
           y_axis_label="Average Runtime per Genome (seconds)",
           toolbar_location=None,
           tools="")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=source,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)